# XL-BOMD
## see https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output = {
'molid': [0,1],
'prefix': 'Outputs/4_XL-BOMD',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params, Temp = 400.0,
              seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 20, remove_com=('angular', 1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   403.00   1.041830e-01 -4.755826e+02 -4.754784e+02 ||    450.02   1.163387e-01 -1.091493e+02 -1.090329e+02 || 
     2   384.43   9.938378e-02 -4.755778e+02 -4.754784e+02 ||    551.44   1.425591e-01 -1.091757e+02 -1.090332e+02 || 
     3   347.72   8.989228e-02 -4.755682e+02 -4.754783e+02 ||    689.19   1.781688e-01 -1.092117e+02 -1.090336e+02 || 
     4   298.52   7.717366e-02 -4.755553e+02 -4.754782e+02 ||    841.38   2.175141e-01 -1.092515e+02 -1.090340e+02 || 
     5   243.75   6.301442e-02 -4.755411e+02 -4.754781e+02 ||    982.13   2.539010e-01 -1.092884e+02 -1.090344e+02 || 
     6   190.40   4.922176e-02 -4.755272e+02 -4.754780e+02 ||   1085.44   2.806075e-01 -1.093154e+02 -1.090348e+02 || 
     7   144.43   3.733759e-02 -4.755152e+02 -4.754779e+02 ||   1129.98   2.921228e-01 -1.093272e+02 -1.090351e+02 || 
     8   109.93   2.842019e-02 -4.755063e+02 -4.754779e+02 ||   1104.04   2.854176e-01 -1.093205e+02 -1.090351e+

## Damping (Langevin)

In [3]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output = {
'molid': [0,1],
'prefix': 'Outputs/4_XL-BOMD_damp',
'print_every': 1,
# write a .xyz file
"xyz": {
    "every": 1    # write .xyz file every N steps; 0 disables
    },
# write an HDF5 file
"h5": {
    "data_every": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "dynamics_vectors_every": 1,   # write vel/forces/coords; 0 disables
    "write_mo": False,
    },
}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100.0,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.24   2.069388e-01 -4.755814e+02 -4.753745e+02 ||    432.66   2.237050e-01 -1.091476e+02 -1.089239e+02 || 
     2   383.18   1.981204e-01 -4.755697e+02 -4.753715e+02 ||    461.50   2.386130e-01 -1.091673e+02 -1.089287e+02 || 
     3   332.17   1.717436e-01 -4.755486e+02 -4.753768e+02 ||    469.43   2.427116e-01 -1.091919e+02 -1.089492e+02 || 
     4   270.12   1.396625e-01 -4.755210e+02 -4.753813e+02 ||    504.32   2.607537e-01 -1.092173e+02 -1.089565e+02 || 
     5   246.87   1.276424e-01 -4.754912e+02 -4.753636e+02 ||    554.88   2.868927e-01 -1.092405e+02 -1.089536e+02 || 
     6   188.83   9.763072e-02 -4.754611e+02 -4.753635e+02 ||    581.81   3.008212e-01 -1.092554e+02 -1.089546e+02 || 
     7   134.46   6.952160e-02 -4.754352e+02 -4.753657e+02 ||    609.08   3.149175e-01 -1.092567e+02 -1.089418e+02 || 
     8   107.47   5.556449e-02 -4.754142e+02 -4.753587e+02 ||    637.80   3.297698e-01 -1.092399e+02 -1.089101e+